In [36]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb
from xgboost import XGBRegressor

import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

In [37]:
df = pd.read_excel("../data/raw/entrenamiento.xlsx")
df = df.rename(columns={"C7.1": "C8", "des": "QUALITY"})
features = df.drop(columns=["QUALITY"])
target = df["QUALITY"]

In [38]:
# Correct the values in column C8 that are above 900 by dividing them by 1000
high_values = df["C8"] > 900
df.loc[high_values, "C8"] = df.loc[high_values, "C8"] / 1000

# Display the basic statistics of the corrected column
df["C8"].describe()

count    3646.000000
mean        0.994044
std         0.003008
min         0.987110
25%         0.991760
50%         0.993800
75%         0.996100
max         1.038980
Name: C8, dtype: float64

In [39]:
# Imputar datos faltantes con la mediana
for col in df.columns:
    median_value = df[col].median()
    df[col].fillna(median_value, inplace=True)

In [40]:
from sklearn.neighbors import LocalOutlierFactor

def impute_outliers_lof(data):
    lof = LocalOutlierFactor(n_neighbors=20, contamination=0.05)
    outliers = lof.fit_predict(data)
    outliers = outliers == -1
    
    # Impute outliers with the median of the entire data (you can adapt this if needed)
    for column in data.columns:
        data.loc[outliers, column] = data[column].median()
    return data

df_imputed_outliers = df.copy()
df_imputed_outliers = impute_outliers_lof(df_imputed_outliers)

In [41]:
X = df_imputed_outliers.drop(columns=['QUALITY'])
y = df_imputed_outliers['QUALITY']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

In [42]:
from xgboost import XGBRegressor

base_learners = [
    ('rf', RandomForestRegressor(max_depth=30,
                                 min_samples_leaf=1,
                                 min_samples_split=2,
                                 n_estimators=100,
                                 bootstrap=True,
                                 random_state=42)),
    ('gboost', GradientBoostingRegressor(learning_rate=0.1,
                                         max_depth=7,
                                         max_features='sqrt',
                                         min_samples_leaf=3,
                                         min_samples_split=2,
                                         n_estimators=100,
                                         subsample=0.9,
                                         random_state=42)),
    ('xgb', XGBRegressor(objective='reg:squarederror',
                         subsample=0.8,
                         min_child_weight=4,
                         max_depth=9,
                         learning_rate=0.1,
                         gamma=0,
                         colsample_bytree=0.8,
                         random_state=42))
]

# Inicializa el modelo de Stacking
stack_reg = StackingRegressor(
    estimators=base_learners,
    final_estimator=LinearRegression(),
    cv=5  # Utiliza validación cruzada con 5 folds para entrenar los modelos base
)

# Train the model
stack_reg.fit(X_train_std, y_train)

# Predict on the training set
stacked_train_predictions = stack_reg.predict(X_train_std)

# Calculate the MAE for the Stacking Regressor
mae_stacked = mean_absolute_error(y_train, stacked_train_predictions)
print(f'MAE of Stacking Regressor: {mae_stacked:.6f}')

MAE of Stacking Regressor: 0.162489


## Delivery

In [43]:
data_validation = pd.read_excel("../data/external/datos_validar_enviar.xlsx")
data_validation = data_validation.rename(columns={"C7.1": "C8"})
data_validation

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11
0,5.9,0.314811,0.277632,1.6,29.00,41.0,4.634729,0.98916,3.41,0.438255,12.7
1,6.1,0.190620,0.173953,1.4,46.00,51.0,4.882802,0.99184,3.22,0.329304,10.5
2,5.8,0.262364,0.113329,1.6,36.00,57.0,5.099866,0.99239,3.38,0.463734,10.5
3,7.4,0.254642,0.405465,1.8,42.00,35.0,4.852030,0.99370,3.45,0.405465,10.2
4,7.4,0.182322,0.978326,2.1,22.00,34.0,4.736198,0.99165,3.26,0.438255,12.2
...,...,...,...,...,...,...,...,...,...,...,...
975,8.7,0.215111,0.300105,0.6,42.00,11.0,4.276666,0.99260,3.08,0.322083,10.6
976,7.6,0.173953,0.343590,1.1,0.04,38.0,4.969813,0.99070,2.92,0.350657,11.4
977,6.2,0.254642,0.207014,12.4,48.00,33.0,5.308268,0.99612,3.11,0.444686,9.9
978,7.5,0.239017,0.307485,7.0,36.00,45.0,5.105945,0.99390,3.03,0.285179,11.0


In [48]:
data_validation.shape

(980, 11)

In [44]:
data_validation.describe().T

,count,mean,std,min,25%,50%,75%,max
C1,980.0,6.850255,0.811157,4.200000,6.300000,6.800000,7.325000,10.700000
C2,980.0,0.389907,0.892588,0.076961,0.190620,0.239017,0.285179,6.666957
C3,980.0,0.286630,0.091586,0.000000,0.237041,0.277632,0.329304,0.978326
C4,980.0,6.381684,5.119473,0.600000,1.700000,4.950000,10.000000,31.600000
C5,980.0,41.000224,25.514386,0.020000,33.000000,42.000000,48.000000,244.000000
C6,980.0,35.312245,16.465520,4.000000,24.000000,34.000000,45.000000,146.500000
C7,980.0,4.891151,0.308470,3.367296,4.700480,4.897840,5.107456,5.731722
C8,980.0,55.740214,226.822839,0.987130,0.991700,0.993900,0.996600,1001.000000
C9,980.0,3.187520,0.145030,2.800000,3.080000,3.180000,3.280000,3.790000
C10,980.0,0.398118,0.074722,0.223144,0.343590,0.392042,0.438255,0.672944


In [45]:
# Correct the values in column C8 that are above 900 by dividing them by 1000
high_values = data_validation["C8"] > 900
data_validation.loc[high_values, "C8"] = data_validation.loc[high_values, "C8"] / 1000

In [46]:
# 1. Transforma el conjunto de datos de validación con el objeto scaler original
data_output_std = scaler.transform(data_validation)

# 2. Usa el modelo de stacking entrenado para predecir los valores de QUALITY
predicted_quality = stack_reg.predict(data_output_std)

In [55]:
predicted_quality

array([6.9277716 , 5.48019908, 5.87376582, 6.37093958, 6.79936313,
       6.3724057 , 5.60940266, 5.71260632, 5.69538381, 6.1508191 ,
       5.76670501, 5.34024493, 6.13482539, 6.19278776, 6.69741233,
       6.29356576, 6.91460203, 6.30267045, 6.15422132, 5.92063078,
       7.16726695, 6.39715229, 5.38550002, 4.98335306, 5.89597816,
       6.67017854, 6.35921123, 6.32100298, 5.31479828, 7.01203119,
       4.28489993, 5.32961547, 5.97606295, 4.97411916, 5.12356465,
       5.33394439, 6.18351856, 6.32902064, 5.54259911, 5.92743198,
       5.22524867, 6.52413959, 6.01591322, 6.34913657, 4.29275161,
       5.8618106 , 5.96776441, 5.22253252, 6.08313581, 5.88564825,
       6.08313581, 6.05547774, 6.54256863, 5.32340039, 5.50324854,
       6.9507303 , 5.37262953, 5.75186933, 5.335485  , 5.94649797,
       6.22521107, 6.22947118, 5.25486699, 5.47296581, 5.91970536,
       5.84308633, 4.78204091, 6.047561  , 5.86501497, 6.10131066,
       6.03192301, 5.00120783, 4.92460851, 6.45002691, 5.03326

In [56]:
import pandas as pd

# Suponiendo que tus predicciones se llaman 'predicted_values'
predicted_values = stack_reg.predict(data_output_std)

# Crear un DataFrame con las predicciones
df_predictions = pd.DataFrame(predicted_values, columns=['Predicted_QUALITY'])

# Guardar el DataFrame en un archivo .xlsx
df_predictions.to_excel('predicted_values.xlsx', index=False)